In [7]:
from langchain.document_loaders import CSVLoader
local_doc_path = 'accidentLabeled.csv'
loader = CSVLoader(file_path=local_doc_path,encoding='utf-8')
docs = loader.load()

In [8]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() 
query="an accident involve multiple cars"
embed = embeddings.embed_query(query)

In [9]:
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(docs, embeddings)
relative_docs = db.similarity_search(query)
relative_docs[:3]

[Document(page_content=': multiple cars\naccident type: This case involves a seriously-injured, belt-restrained, 41-year-old, female driver of a 2012 Nissan Rogue four-door Sport Utility Vehicle, involved in a multi-event-crash. The primary event in this crash involved a collision between the frontal plane of the case vehicle and the frontal plane of a 2005 Ford Focus four-door sedan. Prior to this collision, the case vehicle sustained a far-side sideswipe collision with a 2001 Volvo S60.This four-vehicle collision occurred during the post-midnight hours (dark, street lights present) on a multi-lane urban interstate. This trafficway is composed of asphalt and has seven lanes that primarily travel in an east and west direction and are separated by a Jersey barrier. The eastbound lanes have a negative four percent grade and four travel lanes. The westbound roadway has three travel lanes. The roadway is straight with no traffic control devices. There is a posted speed limit of 113 km/h (7

In [10]:
from langchain.llms import OpenAI
context = "".join([doc.page_content for doc in relative_docs])
prompt=f"""You are an expert in generating accident reports. Please generate accident report about {query}, and here are some examples of relative accident reports: {context}"""

In [11]:
from openai import OpenAI
import os
openai_api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key=openai_api_key)
def get_completion_from_openai(prompt, model='gpt-3.5-turbo'):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.8,
        top_p=0.8,
        max_tokens=512,
        frequency_penalty=1.1,
        presence_penalty=1.1
    )
    return response.choices[0].message.content

In [12]:
from IPython.display import Markdown
import json
json_file = 'accident.json'
try:
    with open(json_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
except (FileNotFoundError, json.JSONDecodeError):
    data = []
for i in range(5):
    print(i)
    response = get_completion_from_openai(prompt=prompt)
    Markdown(response)
    new_data = {"query": query, "response": response}
    data.append(new_data)
    with open(json_file, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

0
1
2
3
4
